In [1]:
import sys
sys.path.append('/home/simon/git/lda/code/')

In [2]:
from ms2lda_feature_extraction import LoadMGF

In [3]:
mgf_file = '/home/simon/Dropbox/MolecularNetworking_GCF-MF_Crusemann_Duncan/Crusemann_only_Clutered_Data/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'

In [4]:
min_ms2_intensity = 25

In [5]:
l = LoadMGF(min_ms2_intensity = 25)
ms1,ms2,metadata = l.load_spectra([mgf_file])

Filtering MS2 on intensity
85977 MS2 remaining


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
mols_as_dicts = {}
for m in ms1:
    mols_as_dicts[m] = []

In [9]:
for f in ms2:
    m = f[3]
    mols_as_dicts[m].append((f[0],f[2]))

In [10]:
for m in mols_as_dicts:
    mols_as_dicts[m] = sorted(mols_as_dicts[m],key = lambda x: x[0])

In [11]:
for m in mols_as_dicts:
    max_val = max([x[1] for x in mols_as_dicts[m]])
    new_spec = []
    for mz,i in mols_as_dicts[m]:
        new_spec.append((mz,100.0*i/max_val))
    mols_as_dicts[m] = new_spec

In [18]:
parentmass = {}
for m in ms1:
    parentmass[m] = metadata[m.name].get('parentmass')

Compute all the scores

In [40]:
scores = []
n_mols = len(ms1)
import numpy as np
scoremat = np.zeros((n_mols,n_mols),np.double)
n_matches = np.zeros((n_mols,n_mols),np.int)

In [41]:
from spectrum_alignment import score_alignment
tol = 0.02
n_done = 0
max_align = 0
max_score = 0
for i,m1 in enumerate(ms1[:-1]):
    for j,m2 in enumerate(ms1[i+1:]):
        score,alignment = score_alignment(mols_as_dicts[m1],mols_as_dicts[m2],parentmass[m1],parentmass[m2],tol)
#         scoremat[i,j] = score
#         n_matches[i,j] = len(alignment)
        if len(alignment) > 0:
            scores.append((i,j,score,len(alignment)))
            if len(alignment) > max_align:
                max_align = len(alignment)
                print max_align
            if score > max_score:
                max_score = score
                print max_score
    n_done += 1
    if n_done % 500 == 0:
        print n_done,n_mols
#         scores.append((m1,m2,score))
    


1
0.0998928382328
0.173722421973
2
0.26802379854
3
0.301117502948
0.469879957612
4
5
0.576708436394
6
0.649296299183
7
0.683601555959
8
9
0.759130709126
0.806750636486
0.947818952788
0.953455183197
11
12
0.983731880113
13
0.98450796307
0.992077415301
500 5930
1000 5930
15
16
0.994420400544
1500 5930
18
2000 5930
22
23
2500 5930
24
25
28
3000 5930
3500 5930
29
4000 5930
4500 5930
5000 5930
5500 5930


In [43]:
share6 = filter(lambda x: x[3]>=6, scores)
print len(share6)

46927


In [44]:
for row in share6:
    i,j,s,a = row
    scoremat[i][j] = s
    scoremat[j][i] = s

In [47]:
from sklearn.cluster import AffinityPropagation

In [54]:
a = AffinityPropagation(affinity = 'precomputed',verbose=True,max_iter = 1000)
labels = a.fit_predict(scoremat)

Did not converge


In [55]:
print len(np.unique(labels))

397


Make a cytoscape output

In [78]:
mols = set()
edges = []
for l in np.unique(labels):
    l_count = (labels == l).sum()
    if l_count > 1:
        pos = np.where(labels==l)[0]
        for i,p in enumerate(pos[:-1]):
            for q in pos[i+1:]:
                if scoremat[p][q] >= 0.5:
                    edges.append((p,q,scoremat[p][q]))


In [79]:
import csv
with open('crusemann_edges.csv','w') as f:
    writer = csv.writer(f)
    for edge in edges:
        writer.writerow(edge)

In [80]:
print share6[0]

(2, 96, 0.6492962991833134, 6)


In [81]:
f = filter(lambda x: x[1]==96,share6)

In [82]:
print f

[(2, 96, 0.6492962991833134, 6), (32, 96, 0.5416920274823939, 7), (291, 96, 0.4431204609785644, 6), (446, 96, 0.5090100719473236, 7), (480, 96, 0.6533412236589695, 6), (494, 96, 0.6085590640866237, 6), (551, 96, 0.96456387638787, 9), (843, 96, 0.5212520505138598, 6), (953, 96, 0.7364467887122442, 6), (1203, 96, 0.5889772997459092, 6), (1457, 96, 0.4818487422340393, 6), (1850, 96, 0.2775405695300927, 6), (2021, 96, 0.9059481447513584, 7), (2022, 96, 0.8543089331043593, 7), (2024, 96, 0.9076675329538069, 6), (2027, 96, 0.9143589289019383, 6), (2046, 96, 0.9211307580077965, 6), (2458, 96, 0.3712788918608551, 7), (3011, 96, 0.7696027777501996, 8), (3327, 96, 0.7315984670494418, 6), (3507, 96, 0.3772879818503447, 6), (3683, 96, 0.6435350958785386, 8), (4152, 96, 0.6020001148276687, 7), (4162, 96, 0.5409685888534158, 6), (4164, 96, 0.6084863295969036, 6), (4168, 96, 0.6487976088510696, 6), (4375, 96, 0.7586138491171733, 7), (4376, 96, 0.19589092445382833, 6), (4588, 96, 0.4367116165489802, 7

In [103]:
from mol_networks import create_edge_dict,top_k_filter
print scores[0]
edge_dict = create_edge_dict(scores,ms1)

(0, 0, 0.0998928382327748, 1)


In [104]:
from mol_networks import create_edge_dict,top_k_filter
filtered_edge_dict,n_edges = top_k_filter(edge_dict)

In [142]:
from mol_networks import make_network
components = make_network(filtered_edge_dict)

1
2
1
1
1
1
2
1
1
1
2
1
1
2
3
1
1
2
1
1
2
1
99
1
1
1
2
1
1
1
3
1
1
17
2
3
1
1
1
2
1
3
96
2
1
4
6
1
1
1
1
1
1
1
3
3
1
8
2
1
1
1
2
3
1
1
5
4
2
1
2
78
4
12
100
1
1
1
9
1
17
3
9
65
1
1
1
71
57
1
1
1
62
1
32
39
12
1
1
59
1
1
4
1
5
8
4
100
3
1
1
1
1
2
13
62
100
84
1
2
15
95
3
1
3
1
1
1
3
1
1
1
1
3
10
59
1
7
37
2
3
3
1
9
1
5
2
3
4
2
1
3
1
1
1
5
2
4
1
2
2
2
1
5
2
8
4
1
2
2
1
1
3
1
2
1
2
8
1
1
4
1
4
1
1
5
1
1
2
1
3
3
1
1
4
2
1
1
1
2
1
1
1
1
1
1
2
2
5
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
2
1
2
1
1
1
3
1
3
2
1
11
2
1
1
1
1
1
9
6
1
2
6
1
2
2
1
1
1
1
1
1
1
1
2
8
1
1
1
1
1
1
3
1
2
1
5
2
1
1
2
1
1
1
2
1
1
1
2
1
2
2
9
1
1
2
4
2
3
1
3
2
2
1
2
2
1
1
1
1
1
1
1
1
1
1
3
1
2
5
1
1
1
2
1
1
2
1
1
1
1
1
4
4
1
1
1
1
1
2
1
1
1
6
1
2
1
1
2
1
1
4
1
1
2
1
1
1
1
2
1
1
2
1
1
1
1
1
2
2
1
2
1
1
1
1
1
1
1
1
5
1
1
1
1
4
1
1
1
1
1
1
1
1
1
6
1
1
1
2
1
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
7
1
1
1
3
2
1
2
1
1
1
1
2
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1

In [146]:
component_thresholds = components[1]
components = components[0]

In [147]:
reverse = {}
for mol,c in components.items():
    if c > -1:
        if not c in reverse:
            reverse[c] = [mol]
        else:
            reverse[c].append(mol)

In [153]:
with open('crusemann_edges.csv','w') as f:
    writer = csv.writer(f)
    for c,mols in reverse.items():
        for i,m in enumerate(mols):
            for j in mols[i+1:]:
                ends,scores = zip(*filtered_edge_dict[m])
                if j in ends:
                    score = scores[ends.index(j)]
                    if score >= component_thresholds[c]:
                        writer.writerow([m,j,score])

In [116]:
print len(filtered_edge_dict)

3806
